In [1]:
import os
os.chdir( "../")

In [2]:
import psutil
num_cpus = psutil.cpu_count(logical=False)
num_cpus = 8


8

In [3]:
import pandas as pd
import numpy as np

path = "Dataset/data_train.csv"
df = pd.read_csv(filepath_or_buffer=path,
                               sep=",",
                               header=1,
                               engine='python',
                               names=['UserID', 'ItemID', 'Interaction'])

metric_to_optimize="MAP"

df

,UserID,ItemID,Interaction
0,1,15,1.0
1,1,16,1.0
2,1,133,1.0
3,1,161,1.0
4,1,187,1.0
...,...,...,...
478724,13024,13605,1.0
478725,13024,13823,1.0
478726,13024,15122,1.0
478727,13024,18185,1.0


In [4]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [5]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

seeds = [
    7315366, 9428914, 5564236, 
    3420043, 9223036, 851434, 3300082, 7899660, 
    2053768, 4305274, 3709256, 8795594, 1418903, 
    7428566, 157832
]

seeds = seeds[:num_cpus]

urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

urm_train_validation = []
urm_test = []
urm_train = []
urm_validation = []

for i in range(len(seeds)):
    a, b = split_train_in_two_percentage_global_sample(urm_all, train_percentage = 0.80,seed=seeds[i])
    c, d = split_train_in_two_percentage_global_sample(a, train_percentage = 0.80,seed=seeds[i])
    urm_train_validation.append(a)
    urm_test.append(b)
    urm_train.append(c)
    urm_validation.append(d)



In [7]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = []
for u in urm_test:
    evaluator_test.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2136 (16.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2128 (16.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2180 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2196 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2202 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2173 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2187 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2162 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [8]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.MatrixFactorization.ImplicitIALSRecommender import ImplicitALSRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
from Recommenders.Recommender_import_list import *
import optuna

In [9]:
# Replace the following with your MySQL database connection details
host = "db-mysql-nyc3-14396-do-user-15539286-0.c.db.ondigitalocean.com"
port = 25060
database_name = "recsys_optuna"
username = "doadmin"
password = "AVNS_EIQ1D-GbDGJWZPsN3Mt"

# Create an SQLAlchemy engine
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}"
storage = optuna.storages.RDBStorage(url=mysql_url)

nmf_name = "NMF8-FOLD"
knn_name = "itemKNN8-FOLD"
slim_name = "SLIM_EN8-FOLD"
rp3beta_name = "RP3b8-FOLD"
p3alpha_name = "P3a8-FOLD"
hybrid_name = "HYBRID8-FOLD"
#nmf_study = optuna.load_study(study_name=nmf_name, storage=storage)
#knn_study = optuna.load_study(study_name=knn_name, storage=storage)
#slim_study = optuna.load_study(study_name=slim_name, storage=storage)
#rp3beta_study = optuna.load_study(study_name=rp3beta_name, storage=storage)
#p3alpha_study = optuna.load_study(study_name=p3alpha_name, storage=storage)
#hybrid_study = optuna.load_study(study_name=hybrid_name, storage=storage)

#studies = [nmf_study, knn_study, slim_study, rp3beta_study, p3alpha_study]
#studies = [knn_study, slim_study, rp3beta_study, p3alpha_study]
#recommenders_list = ["Hybrid", "itemKNN", "RP3beta", "P3alpha", "NMF", "SLIM_EN", "TopPop"]
    

In [10]:
metric_to_optimize = "MAP"

In [ ]:
from Recommenders.EASE_R.EASE_R_RecommenderV2 import EASE_R_RecommenderV2

In [37]:
ials_recommenders = []
p3a_recomennders = []
rp3b_recomennders = []
slimen_recommenders = []
knn_recommenders = []
slim_recommenders = []
ease_recommenders = []

for i in range(len(seeds)):
    ials_single = ImplicitALSRecommender(urm_train_validation[i])
    ials_single.fit(use_gpu=True, num_threads=1,**{'num_factors': 401, 'reg': 0.0010579931024405477,'epsilon':0.10666578585393433, 'epochs':1204, 'alpha': 7.139858098793055, 'confidence_scaling': 'linear'})
    ials_recommenders.append(ials_single)

    p3a_single = P3alphaRecommender(urm_train_validation[i])
    p3a_single.fit(**{'alpha':0.740243876372529,'topK':41})
    p3a_recomennders.append(p3a_single)

    rp3beta = RP3betaRecommender(urm_train_validation[i])
    rp3beta.fit(**{'topK': 29, 'alpha': 0.33723025040409343, 'beta': 0.15542352567862933, 'normalize_similarity': True})
    rp3b_recomennders.append(rp3beta)

    knn_single = ItemKNNCFRecommender(urm_train_validation[i])
    knn_single.fit(**{'topK': 19, 'shrink': 28, 'similarity': 'tversky', 'normalize': True, 'tversky_alpha': 0.0, 'tversky_beta': 1.3672226785339947})
    knn_recommenders.append(knn_single)

    slim_single = MultiThreadSLIM_SLIMElasticNetRecommender(urm_train_validation[i])
    slim_single.fit(workers=12,**{'alpha':0.0008361537677496276,'topK':3334, 'l1_ratio':0.09976758088492818})
    slim_recommenders.append(slim_single)

    ease_single = EASE_R_RecommenderV2(urm_train_validation[i])
    ease_single.fit(**{'topK': 46, 'normalize_matrix': False, 'l2_norm': 32.19549584425843})
    ease_recommenders.append(ease_single)

NMFRecommender: URM Detected 226 ( 1.8%) users with no interactions.
NMFRecommender: URM Detected 90 ( 0.4%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 16.98 sec
P3alphaRecommender: URM Detected 226 ( 1.8%) users with no interactions.
P3alphaRecommender: URM Detected 90 ( 0.4%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 9524.94 column/sec. Elapsed time 2.33 sec
RP3betaRecommender: URM Detected 226 ( 1.8%) users with no interactions.
RP3betaRecommender: URM Detected 90 ( 0.4%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 8664.90 column/sec. Elapsed time 2.56 sec
ItemKNNCFRecommender: URM Detected 226 ( 1.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 90 ( 0.4%) items with no interactions.
Similarity column 22222 (100.0%), 12704.91 column/sec. Elapsed time 1.75 sec
SLIMElasticNetRecommender: URM Detected 226

100%|█████████▉| 22216/22222 [01:02<00:00, 354.18it/s]


NMFRecommender: URM Detected 240 ( 1.9%) users with no interactions.
NMFRecommender: URM Detected 111 ( 0.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 18.00 sec
P3alphaRecommender: URM Detected 240 ( 1.9%) users with no interactions.
P3alphaRecommender: URM Detected 111 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 9077.90 column/sec. Elapsed time 2.45 sec
RP3betaRecommender: URM Detected 240 ( 1.9%) users with no interactions.
RP3betaRecommender: URM Detected 111 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 8526.15 column/sec. Elapsed time 2.61 sec
ItemKNNCFRecommender: URM Detected 240 ( 1.9%) users with no interactions.
ItemKNNCFRecommender: URM Detected 111 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 12348.45 column/sec. Elapsed time 1.80 sec
SLIMElasticNetRecommender: URM Detected

100%|█████████▉| 22216/22222 [01:03<00:00, 350.06it/s]


NMFRecommender: URM Detected 198 ( 1.6%) users with no interactions.
NMFRecommender: URM Detected 118 ( 0.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 23.10 sec
P3alphaRecommender: URM Detected 198 ( 1.6%) users with no interactions.
P3alphaRecommender: URM Detected 118 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 9337.66 column/sec. Elapsed time 2.38 sec
RP3betaRecommender: URM Detected 198 ( 1.6%) users with no interactions.
RP3betaRecommender: URM Detected 118 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 8556.66 column/sec. Elapsed time 2.60 sec
ItemKNNCFRecommender: URM Detected 198 ( 1.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 118 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 12812.25 column/sec. Elapsed time 1.73 sec
SLIMElasticNetRecommender: URM Detected

...

100%|█████████▉| 22216/22222 [01:04<00:00, 345.96it/s]


NMFRecommender: URM Detected 216 ( 1.7%) users with no interactions.
NMFRecommender: URM Detected 115 ( 0.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 26.44 sec
P3alphaRecommender: URM Detected 216 ( 1.7%) users with no interactions.
P3alphaRecommender: URM Detected 115 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 9306.93 column/sec. Elapsed time 2.39 sec
RP3betaRecommender: URM Detected 216 ( 1.7%) users with no interactions.
RP3betaRecommender: URM Detected 115 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 8588.53 column/sec. Elapsed time 2.59 sec
ItemKNNCFRecommender: URM Detected 216 ( 1.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 115 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 12749.53 column/sec. Elapsed time 1.74 sec
SLIMElasticNetRecommender: URM Detected

100%|█████████▉| 22216/22222 [01:04<00:00, 344.49it/s]


NMFRecommender: URM Detected 216 ( 1.7%) users with no interactions.
NMFRecommender: URM Detected 108 ( 0.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 21.90 sec
P3alphaRecommender: URM Detected 216 ( 1.7%) users with no interactions.
P3alphaRecommender: URM Detected 108 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 9249.61 column/sec. Elapsed time 2.40 sec
RP3betaRecommender: URM Detected 216 ( 1.7%) users with no interactions.
RP3betaRecommender: URM Detected 108 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 8628.85 column/sec. Elapsed time 2.58 sec
ItemKNNCFRecommender: URM Detected 216 ( 1.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 108 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 12717.14 column/sec. Elapsed time 1.75 sec
SLIMElasticNetRecommender: URM Detected

100%|█████████▉| 22216/22222 [01:03<00:00, 349.38it/s]


In [38]:
from Recommenders.Hybrid.GeneralizedLinearHybridRecommender import GeneralizedLinearHybridRecommender

hybrids = []

for i in range(len(seeds)):
    hybrid = GeneralizedLinearHybridRecommender(urm_train_validation[i], recommenders=[ials_recommenders[i], p3a_recomennders[i], rp3b_recomennders[i], knn_recommenders[i], slim_recommenders[i], ease_recommenders[i]], verbose=False)
    hybrids.append(hybrid)

NMFP3alphaRP3betaItemKNNCFSLIMElasticNetHybridRecommender: URM Detected 226 ( 1.8%) users with no interactions.
NMFP3alphaRP3betaItemKNNCFSLIMElasticNetHybridRecommender: URM Detected 90 ( 0.4%) items with no interactions.
NMFP3alphaRP3betaItemKNNCFSLIMElasticNetHybridRecommender: URM Detected 240 ( 1.9%) users with no interactions.
NMFP3alphaRP3betaItemKNNCFSLIMElasticNetHybridRecommender: URM Detected 111 ( 0.5%) items with no interactions.
NMFP3alphaRP3betaItemKNNCFSLIMElasticNetHybridRecommender: URM Detected 198 ( 1.6%) users with no interactions.
NMFP3alphaRP3betaItemKNNCFSLIMElasticNetHybridRecommender: URM Detected 118 ( 0.5%) items with no interactions.
NMFP3alphaRP3betaItemKNNCFSLIMElasticNetHybridRecommender: URM Detected 216 ( 1.7%) users with no interactions.
NMFP3alphaRP3betaItemKNNCFSLIMElasticNetHybridRecommender: URM Detected 121 ( 0.5%) items with no interactions.
NMFP3alphaRP3betaItemKNNCFSLIMElasticNetHybridRecommender: URM Detected 189 ( 1.5%) users with no interac

In [40]:
hybrid_name = "HYBRID_BIGGER_8-FOLD"
hybrid_study = optuna.create_study(study_name=hybrid_name, direction="maximize", storage=storage, load_if_exists=True, sampler=optuna.samplers.TPESampler())

def objective(trial):

    params = {
        "w_ials": trial.suggest_float("w_ials", 0, 1),
        "w_p3a": trial.suggest_float("w_p3a", 0, 1),
        "w_rp3b": trial.suggest_float("w_rp3b", 0, 1),
        "w_knn": trial.suggest_float("w_knn", 0, 1),
        "w_slim": trial.suggest_float("w_slim", 0, 1),
        "w_ease": trial.suggest_float("w_ease", 0, 1)
    }

    alphas = [params["w_ials"], params["w_p3a"], params["w_rp3b"], params["w_knn"], params["w_slim"], params["w_ease"]]

    MAP = 0

    for i in range(len(seeds)):
        hybrids[i].fit(alphas=alphas)
        MAP += evaluator_test[i].evaluateRecommender(hybrids[i])[0][metric_to_optimize]

    average_score = MAP / len(seeds)
    return average_score

#hybrid_study.enqueue_trial({"w_nmf": 0.0, "w_p3a": 0.0, "w_rp3b": 0.0, "w_knn": 0.0, "w_slim": 0.0})
hybrid_study.optimize(objective, n_trials=250)

[I 2024-01-09 11:16:20,486] Using an existing study with name 'HYBRID8-FOLD' instead of creating a new one.


EvaluatorHoldout: Processed 10502 (100.0%) in 6.88 sec. Users per second: 1526
EvaluatorHoldout: Processed 10510 (100.0%) in 6.96 sec. Users per second: 1511
EvaluatorHoldout: Processed 10458 (100.0%) in 7.02 sec. Users per second: 1490
EvaluatorHoldout: Processed 10442 (100.0%) in 7.16 sec. Users per second: 1459
EvaluatorHoldout: Processed 10436 (100.0%) in 6.88 sec. Users per second: 1517
EvaluatorHoldout: Processed 10465 (100.0%) in 6.87 sec. Users per second: 1524
EvaluatorHoldout: Processed 10451 (100.0%) in 6.82 sec. Users per second: 1532
EvaluatorHoldout: Processed 10476 (100.0%) in 6.94 sec. Users per second: 1510


[I 2024-01-09 11:17:31,443] Trial 1 finished with value: 0.048583464892794734 and parameters: {'w_nmf': 0.9856706682953644, 'w_p3a': 0.9833593696535404, 'w_rp3b': 0.9944974998206771, 'w_knn': 0.4521556673572107, 'w_slim': 0.5765019249172418}. Best is trial 1 with value: 0.048583464892794734.


EvaluatorHoldout: Processed 10502 (100.0%) in 6.76 sec. Users per second: 1553
EvaluatorHoldout: Processed 10510 (100.0%) in 6.89 sec. Users per second: 1524
EvaluatorHoldout: Processed 10458 (100.0%) in 6.84 sec. Users per second: 1528
EvaluatorHoldout: Processed 10442 (100.0%) in 7.04 sec. Users per second: 1483
EvaluatorHoldout: Processed 10436 (100.0%) in 6.76 sec. Users per second: 1543
EvaluatorHoldout: Processed 10465 (100.0%) in 7.04 sec. Users per second: 1487
EvaluatorHoldout: Processed 10451 (100.0%) in 6.94 sec. Users per second: 1505
EvaluatorHoldout: Processed 10476 (100.0%) in 7.15 sec. Users per second: 1464


[I 2024-01-09 11:18:40,516] Trial 2 finished with value: 0.048931942748165425 and parameters: {'w_nmf': 0.5187126088431594, 'w_p3a': 0.8914250494714231, 'w_rp3b': 0.8248239219828833, 'w_knn': 0.6655677693009491, 'w_slim': 0.1964614097074876}. Best is trial 2 with value: 0.048931942748165425.


...

EvaluatorHoldout: Processed 10502 (100.0%) in 6.81 sec. Users per second: 1543
EvaluatorHoldout: Processed 10510 (100.0%) in 6.91 sec. Users per second: 1522
EvaluatorHoldout: Processed 10458 (100.0%) in 6.83 sec. Users per second: 1531
EvaluatorHoldout: Processed 10442 (100.0%) in 6.93 sec. Users per second: 1507
EvaluatorHoldout: Processed 10436 (100.0%) in 6.86 sec. Users per second: 1522
EvaluatorHoldout: Processed 10465 (100.0%) in 6.98 sec. Users per second: 1499
EvaluatorHoldout: Processed 10451 (100.0%) in 6.95 sec. Users per second: 1504
EvaluatorHoldout: Processed 10476 (100.0%) in 7.01 sec. Users per second: 1494


[I 2024-01-09 11:35:52,296] Trial 17 finished with value: 0.04696221078173296 and parameters: {'w_nmf': 0.6942944908536635, 'w_p3a': 0.14641345246451312, 'w_rp3b': 0.2808412426429666, 'w_knn': 0.3101088105241165, 'w_slim': 0.33678503054007564}. Best is trial 5 with value: 0.05147424928210691.


EvaluatorHoldout: Processed 10502 (100.0%) in 7.05 sec. Users per second: 1491
EvaluatorHoldout: Processed 10510 (100.0%) in 7.13 sec. Users per second: 1475
EvaluatorHoldout: Processed 10458 (100.0%) in 7.05 sec. Users per second: 1484
EvaluatorHoldout: Processed 10442 (100.0%) in 6.99 sec. Users per second: 1495
EvaluatorHoldout: Processed 10436 (100.0%) in 6.94 sec. Users per second: 1504
EvaluatorHoldout: Processed 10465 (100.0%) in 7.04 sec. Users per second: 1487
EvaluatorHoldout: Processed 10451 (100.0%) in 6.97 sec. Users per second: 1499
EvaluatorHoldout: Processed 10476 (100.0%) in 7.07 sec. Users per second: 1483


[I 2024-01-09 11:37:01,988] Trial 18 finished with value: 0.05076651648536702 and parameters: {'w_nmf': 0.10873878293892347, 'w_p3a': 0.568588237384287, 'w_rp3b': 0.46263753471465013, 'w_knn': 0.29824287278202594, 'w_slim': 0.6038347234959351}. Best is trial 5 with value: 0.05147424928210691.


EvaluatorHoldout: Processed 10502 (100.0%) in 6.84 sec. Users per second: 1535
EvaluatorHoldout: Processed 10510 (100.0%) in 6.91 sec. Users per second: 1520
